
**Traning by using MobileNet**












In [ ]:
#Import tensorflow into colab
import tensorflow as tf
print("TF version:",tf.__version__)
#Check for GPU availability
print("GPU", "available (YESSSSS!!!!!!)" if tf.config.list_physical_devices("GPU") else "not available")

In [ ]:
#chekking total training images
dam_path='/content/drive/MyDrive/colab_notes/car_damage_vision/car_damage_data/training/00-damage'
not_dam_path='/content/drive/MyDrive/colab_notes/car_damage_vision/car_damage_data/training/01-whole'
total_training_img=(len(dam_path)+ len(not_dam_path))
print(f'nomber of total training image :{total_training_img}')
print(f'nomber of not damaged image : {len(not_dam_path)}')
print(f'nomber of damaged image :{len(dam_path)}')

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize = (10, 5))
 
# creating the bar plot
plt.bar(len(dam_path), len(dam_path), color ='maroon',
        width = 0.4)
plt.bar(len(not_dam_path), len(not_dam_path), color ='yellow',
        width = 0.4)
plt.xlabel("not damaged images")
plt.ylabel("damaged images")
plt.title("cound of training images")
plt.show()


In [ ]:
# just showing a damaged car image
from IPython.display import Image
Image('/content/drive/MyDrive/colab_notes/car_damage_vision/car_damage_data/training/00-damage/0036.JPEG')

In [ ]:
pip install --upgrade imutils

In [ ]:
# pip install imutils. Released: Jan 15, 2021. A series of convenience functions to make basic
# image processing functions such as translation, rotation, resizing, skeletonization, displaying 
# Matplotlib images, sorting contours, detecting edges, and much more easier with OpenCV and both 
# Python 2.7 and Python 3.
# pip install --upgrade imutils

In [ ]:
# necessary imports
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# Keras ImageDataGenerator is a gem! It lets you augment your images in real-time while your
# model is still training! You can apply any random transformations on each training image as 
# it is passed to the model. This will not only make your model robust but will also save up on 
# the overhead memory!
print('Image Augmentation on the fly using Keras ImageDataGenerator!')
import cv2
im = cv2.imread('/content/drive/MyDrive/colab_notes/car_damage_vision/Screenshot (22).png')
# im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
plt.axis("off")
plt.imshow(im)


from tensorflow.keras.applications import MobileNetV2
# MobileNet is a CNN architecture that is much faster as well as a smaller model that makes use 
# of a new kind of convolutional layer, known as Depthwise Separable convolution. Because of the 
# small size of the model, these models are considered very useful to be implemented on mobile 
# and embedded devices.

from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
# mobilenet_v2. preprocess_input will scale input pixels between -1 and 1. Arguments. 
# input_shape: Optional shape tuple, to be specified if you would like to use a model with 
# an input image resolution that is not (224, 224, 3). It should have exactly 3 inputs channels (224, 224, 3).

from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths




In [ ]:
# initialize the initial learning rate, number of epochs to train for,
# and batch size
EPOCHS = 50
BS = 64

Training_data_path = r"/content/drive/MyDrive/colab_notes/car_damage_vision/car_damage_data/training"
Validation_data_path=r"/content/drive/MyDrive/colab_notes/car_damage_vision/car_damage_data/validation"
CATEGORIES = ["00-damage", "01-whole"]

# grab the list of images in our dataset directory, then initialize
# the list of data (i.e., images) and class images
print("[INFO] loading images...")

data = []
labels = []

for category in CATEGORIES:
    path = os.path.join(Training_data_path, category)
    for img in os.listdir(path):
    	img_path = os.path.join(path, img)
    	image = load_img(img_path, target_size=(224, 224))
    	image = img_to_array(image)
    	image = preprocess_input(image)

    	data.append(image)
    	labels.append(category)


for category in CATEGORIES:
    path = os.path.join(Validation_data_path, category)
    for img in os.listdir(path):
    	img_path = os.path.join(path, img)
    	image = load_img(img_path, target_size=(224, 224))
    	image = img_to_array(image)
    	image = preprocess_input(image)

    	data.append(image)
    	labels.append(category)
        
# perform one-hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

data = np.array(data, dtype="float32")
labels = np.array(labels)



(trainX, testX, trainY, testY) = train_test_split(data, labels,
	test_size=0.20, stratify=labels, random_state=42)

# construct the training image generator for data augmentation
aug = ImageDataGenerator(
	rotation_range=20,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")



In [ ]:

from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# load the MobileNetV2 network, ensuring the head FC layer sets are
# left off
baseModel = MobileNetV2(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))

# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = MaxPooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel.layers:
	layer.trainable = False

# compile our model
INIT_LR = 1e-5
print("[INFO] compiling model...")
# lr=INIT_LR, decay=INIT_LR / EPOCHS
opt = Adam()
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])


# train the head of the network
print("[INFO] training head...")
H = model.fit(
	aug.flow(trainX, trainY, batch_size=BS),
	steps_per_epoch=len(trainX) // BS,
	validation_data=(testX, testY),
	validation_steps=len(testX) // BS,
	epochs=EPOCHS)

# make predictions on the testing set
print("[INFO] evaluating network...")
predIdxs = model.predict(testX, batch_size=BS)

# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)

# show a nicely formatted classification report
print(classification_report(testY.argmax(axis=1), predIdxs,
	target_names=lb.classes_))

# serialize the model to disk
print("[INFO] saving mask detector model...")
model.save("MobileNet_Car_detection.model", save_format="h5")

# plot the training loss and accuracy
N = EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("MobileNet.png")

In [ ]:
# model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
from keras import models    
MobileNet = models.load_model("MobileNet_Car_detection.model")

In [ ]:
testURL="/content/drive/MyDrive/colab_notes/car_damage_vision/car_damage_data/validation"
damage_test_url="/content/drive/MyDrive/colab_notes/car_damage_vision/car_damage_data/validation/00-damage"
whole_test_url="/content/drive/MyDrive/colab_notes/car_damage_vision/car_damage_data/validation/01-whole"
damage_img = [f'{damage_test_url}/{i}' for i in os.listdir(damage_test_url)]
whole_img=[f'{whole_test_url}/{i}' for i in os.listdir(whole_test_url)]
total_img=damage_img+whole_img

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np
import cv2
import matplotlib.pyplot as plt
img_path = "/content/drive/MyDrive/colab_notes/car_damage_vision/car_damage_data/validation/01-whole/0008.jpg"
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_batch = np.expand_dims(img_array, axis=0)


im = cv2.imread(img_path)
im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
plt.axis("off")
plt.imshow(im)

pred=MobileNet.predict(img_batch)
print("Car is Damaged: "+str(pred[0][0])+", Car is not Damaged: "+str(pred[0][1]))
if pred[0][0]<pred[0][1]:
    print("The car is not damaged")
else:
    print("The car is damaged")

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
img_path = "/content/drive/MyDrive/colab_notes/car_damage_vision/car_damage_data/validation/00-damage/0017.JPEG"
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_batch = np.expand_dims(img_array, axis=0)


im = cv2.imread(img_path)
im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
plt.axis("off")
plt.imshow(im)

pred=MobileNet.predict(img_batch)
print("Car is Damaged: "+str(pred[0][0])+", Car is not Damaged: "+str(pred[0][1]))
if pred[0][0]<pred[0][1]:
    print("The car is not damaged")
else:
    print("The car is damaged")